(working_with_InferenceData)=

# Working with InferenceData

Here we present a collection of common manipulations you can use while working with `InferenceData`.

In [1]:
import arviz as az
import numpy as np
import xarray as xr
xr.set_options(display_expand_data=False, display_expand_attrs=False);

`display_expand_data=False` makes the default view for {class}`xarray.DataArray` fold the data values to a single line. To explore the values, click on the {fas}`database` icon on the left of the view, right under the `xarray.DataArray` text. It has no effect on `Dataset` objects that already default to folded views.

`display_expand_attrs=False` folds the attributes in both `DataArray` and `Dataset` objects to keep the views shorter. In this page we print DataArrays and Datasets several times and they always have the same attributes.

In [2]:
idata = az.load_arviz_data("centered_eight")
idata

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

## Get the dataset corresponding to a single group

In [3]:
post = idata.posterior
post

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 -3.477 -2.456 -2.826 ... 4.597 5.899 0.1614
    theta    (chain, draw, school) float64 1.669 -8.537 -2.623 ... 10.59 4.523
    tau      (chain, draw) float64 3.73 2.075 3.703 4.146 ... 8.346 7.711 5.407
Attributes: (3)

:::{tip} 
You'll have noticed we stored the posterior group in a new variable: `post`. As `.copy()` was not called, now using `idata.posterior` or `post` is equivalent.

Use this to keep your code short yet easy to read. Store the groups you'll need very often as separate variables to use explicitly, but don't delete the InferenceData parent. You'll need it for many ArviZ functions to work properly. For example: {func}`~arviz.plot_pair` needs data from `sample_stats` group to show divergences, {func}`~arviz.compare` needs data from both `log_likelihood` and `posterior` groups, {func}`~arviz.plot_loo_pit` needs not 2 but 3 groups: `log_likelihood`, `posterior_predictive` and `posterior`.
:::

## Add a new variable


In [4]:
post["log_tau"] = np.log(post["tau"])
idata.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 -3.477 -2.456 -2.826 ... 4.597 5.899 0.1614
    theta    (chain, draw, school) float64 1.669 -8.537 -2.623 ... 10.59 4.523
    tau      (chain, draw) float64 3.73 2.075 3.703 4.146 ... 8.346 7.711 5.407
    log_tau  (chain, draw) float64 1.316 0.7301 1.309 ... 2.122 2.043 1.688
Attributes: (3)

## Combine chains and draws

In [5]:
stacked = az.extract_dataset(idata)
stacked

<xarray.Dataset>
Dimensions:  (school: 8, sample: 2000)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
  * sample   (sample) MultiIndex
  - chain    (sample) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3
  - draw     (sample) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
Data variables:
    mu       (sample) float64 -3.477 -2.456 -2.826 -1.996 ... 4.597 5.899 0.1614
    theta    (school, sample) float64 1.669 -6.239 2.195 ... -1.095 4.013 4.523
    tau      (sample) float64 3.73 2.075 3.703 4.146 ... 8.589 8.346 7.711 5.407
    log_tau  (sample) float64 1.316 0.7301 1.309 1.422 ... 2.122 2.043 1.688
Attributes: (3)

You can also use {meth}`xarray.Dataset.stack` if you only want to combine the chain and draw dimensions. {func}`arviz.extract_dataset` is a convenience function aimed at taking care of the most common subsetting operations with MCMC samples. It can:
- Combine chains and draws
- Return a subset of variables (with optional filtering with regular expressions or string matching)
- Return a subset of samples. Moreover by default it returns a random subset to prevent getting non-representative samples due to bad mixing.
- Acess any group

(idata/random_subset)=
## Get a random subset of the samples

In [6]:
az.extract_dataset(idata, num_samples=100)

<xarray.Dataset>
Dimensions:  (school: 8, sample: 100)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
  * sample   (sample) MultiIndex
  - chain    (sample) int64 0 0 0 3 1 2 2 2 3 2 1 0 ... 3 3 3 1 1 0 1 1 1 1 2 3
  - draw     (sample) int64 419 274 161 193 178 203 ... 238 49 212 448 95 412
Data variables:
    mu       (sample) float64 6.95 7.4 4.131 1.644 ... 5.142 1.917 8.56 3.486
    theta    (school, sample) float64 8.734 3.829 19.1 ... 3.086 11.3 3.606
    tau      (sample) float64 1.867 1.603 8.83 1.929 ... 1.636 5.707 5.941 1.582
    log_tau  (sample) float64 0.6243 0.4717 2.178 0.6572 ... 1.742 1.782 0.4588
Attributes: (3)

:::{tip}
Use a random seed to get the same subset from multiple groups: `az.extract_dataset(idata, num_samples=100, rng=3)` and `az.extract_dataset(idata, group="log_likelihood", num_samples=100, rng=3)` will continue to have matching samples
:::

## Obtain a NumPy array for a given parameter

Let's say we want to get the values for `mu` as a NumPy array.

In [7]:
stacked.mu.values

array([-3.47698606, -2.45587061, -2.82625433, ...,  4.59705819,
        5.89850592,  0.16138927])

## Get the dimension lengths

Let’s check how many groups are in our hierarchical model.

In [8]:
len(idata.observed_data.school)

8

## Get coordinate values

What are the names of the groups in our hierarchical model? You can access them from the coordinate name `school` in this case

In [9]:
idata.observed_data.school

<xarray.DataArray 'school' (school: 8)>
'Choate' 'Deerfield' 'Phillips Andover' ... "St. Paul's" 'Mt. Hermon'
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'

## Get a subset of chains

Let’s keep only chain 0 and 2 here. For the subset to take effect on all relevant InferenceData groups: posterior, sample_stats, log_likelihood, posterior_predictive we will use the {meth}`arviz.InferenceData.sel`, the method of InferenceData instead of {meth}`xarray.Dataset.sel`.

In [10]:
idata.sel(chain=[0, 2])

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

## Remove the first n draws (burn-in)

Let’s say we want to remove the first 100 samples, from all the chains and all `InferenceData` groups with draws.

In [11]:
idata.sel(draw=slice(100, None))

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

If you check the `burnin` object you will see that the groups `posterior`, `posterior_predictive`, `prior` and `sample_stats` have 400 draws compared to `idata` that has 500. The group `observed_data` has not been affected because it does not have the `draw` dimension. Alternatively, you can specify which group or groups you want to change.

In [12]:
idata.sel(draw=slice(100, None), groups="posterior")

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

## Compute posterior mean values along `draw` and `chain` dimensions

To compute the mean value of the posterior samples, do the following:


In [13]:
post.mean()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    mu       float64 4.093
    theta    float64 4.56
    tau      float64 4.089
    log_tau  float64 1.15

This computes the mean along all dimensions. This is probably what you want for `mu` and `tau`, which have two dimensions (`chain` and `draw`), but maybe not what you expected for `theta`, which has one more dimension `school`. 

You can specify along which dimension you want to compute the mean (or other functions).

In [14]:
post.mean(dim=['chain', 'draw'])

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       float64 4.093
    theta    (school) float64 6.026 4.724 3.576 4.478 3.064 3.821 6.25 4.544
    tau      float64 4.089
    log_tau  float64 1.15

## Compute and store posterior pushforward quantities

We use "posterior pushfoward quantities" to refer to quantities that are not variables in the posterior but deterministic computations using posterior variables. 

You can use xarray for these pushforward operations and store them as a new variable in the posterior group. You'll then be able to plot them with ArviZ functions, calculate stats and diagnostics on them (like the {func}`~arviz.mcse`) or save and share the inferencedata object with the pushforward quantities included. 

Compute the rolling mean of $\log(\tau)$ with {meth}`xarray.DataArray.rolling`, storing the result in the posterior

In [15]:
post["mlogtau"] = post["log_tau"].rolling({'draw': 50}).mean()

Using xarray for pusforward calculations has all the advantages of working with xarray. It also inherits the disadvantages of working with xarray, but we believe those to be outweighed by the advantages, and we have already shown how to extract the data as NumPy arrays. Working with InferenceData is working mainly with xarray objects and this is what is shown in this guide.

Some examples of these advantages are specifying operations with named dimensions instead of positional ones (as seen in some previous sections), 
automatic alignment and broadcasting of arrays (as we'll see now),
or integration with Dask (as shown in the {ref}`dask_for_arviz` guide).

In this cell you will compute pairwise differences between schools on their mean effects (variable `theta`).
To do so, substract the variable theta after renaming the school dimension to the original variable. 
Xarray then aligns and broadcasts the two variables because they have different dimensions, and
the result is a 4d variable with all the pointwise differences.

Eventually, store the result in the `theta_school_diff` variable:

In [16]:
post['theta_school_diff'] = post.theta - post.theta.rename(school="school_bis")

:::{note}
:class: dropdown

This same operation using NumPy would require manual alignment of the two arrays to make sure they broadcast correctly. The could would be something like:

```python
theta_school_diff = theta[:, :, :, None] - theta[:, :, None, :]
```
:::

The `theta_shool_diff` variable in the posterior has kept the named dimensions and coordinates:

In [17]:
post

<xarray.Dataset>
Dimensions:            (chain: 4, draw: 500, school: 8, school_bis: 8)
Coordinates:
  * chain              (chain) int64 0 1 2 3
  * draw               (draw) int64 0 1 2 3 4 5 6 ... 494 495 496 497 498 499
  * school             (school) object 'Choate' 'Deerfield' ... 'Mt. Hermon'
  * school_bis         (school_bis) object 'Choate' 'Deerfield' ... 'Mt. Hermon'
Data variables:
    mu                 (chain, draw) float64 -3.477 -2.456 ... 5.899 0.1614
    theta              (chain, draw, school) float64 1.669 -8.537 ... 4.523
    tau                (chain, draw) float64 3.73 2.075 3.703 ... 7.711 5.407
    log_tau            (chain, draw) float64 1.316 0.7301 1.309 ... 2.043 1.688
    mlogtau            (chain, draw) float64 nan nan nan ... 0.9753 1.004 1.034
    theta_school_diff  (chain, draw, school, school_bis) float64 0.0 ... 0.0
Attributes: (3)

## Advanced subsetting
To select the value corresponding to the difference between the Choate and Deerfield schools do:

In [18]:
post['theta_school_diff'].sel(school="Choate", school_bis="Deerfield")

<xarray.DataArray 'theta_school_diff' (chain: 4, draw: 500)>
10.21 -7.311 5.116 2.606 -1.116 24.96 ... 3.128 -4.62 4.288 2.424 2.613 -0.1137
Coordinates:
  * chain       (chain) int64 0 1 2 3
  * draw        (draw) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
    school      <U6 'Choate'
    school_bis  <U9 'Deerfield'

For more advanced subsetting (the equivalent to what is sometimes called "fancy indexing" in NumPy) you need to provide the indices as {class}`~xarray.DataArray` objects:

In [19]:
school_idx = xr.DataArray(["Choate", "Hotchkiss", "Mt. Hermon"], dims=["pairwise_school_diff"])
school_bis_idx = xr.DataArray(["Deerfield", "Choate", "Lawrenceville"], dims=["pairwise_school_diff"])
post['theta_school_diff'].sel(school=school_idx, school_bis=school_bis_idx)

<xarray.DataArray 'theta_school_diff' (chain: 4, draw: 500,
                                       pairwise_school_diff: 3)>
10.21 -5.673 2.356 -7.311 2.817 -1.51 ... 2.613 8.154 8.915 -0.1137 2.805 5.63
Coordinates:
  * chain       (chain) int64 0 1 2 3
  * draw        (draw) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
    school      (pairwise_school_diff) object 'Choate' 'Hotchkiss' 'Mt. Hermon'
    school_bis  (pairwise_school_diff) object 'Deerfield' ... 'Lawrenceville'
Dimensions without coordinates: pairwise_school_diff

Using lists or NumPy arrays instead of DataArrays does colum/row based indexing. As you can see, the result has 9 values of `theta_shool_diff` instead of the 3 pairs of difference we selected in the previous cell:

In [20]:
post['theta_school_diff'].sel(
    school=["Choate", "Hotchkiss", "Mt. Hermon"], 
    school_bis=["Deerfield", "Choate", "Lawrenceville"]
)

<xarray.DataArray 'theta_school_diff' (chain: 4, draw: 500, school: 3,
                                       school_bis: 3)>
10.21 0.0 10.84 4.533 -5.673 5.169 1.719 ... 2.691 2.805 3.861 4.46 4.574 5.63
Coordinates:
  * chain       (chain) int64 0 1 2 3
  * draw        (draw) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
  * school      (school) object 'Choate' 'Hotchkiss' 'Mt. Hermon'
  * school_bis  (school_bis) object 'Deerfield' 'Choate' 'Lawrenceville'

## Add new chains using concat

After checking the {func}`~arviz.mcse` and realizing you need more samples, you rerun the model with two chains
and obtain an `idata_rerun` object.

In [21]:
idata_rerun = idata.sel(chain=[0, 1]).copy().assign_coords(coords={"chain":[4,5]},groups="posterior_groups")

You can combine the two into a single InferenceData object using {func}`arviz.concat`:

In [22]:
idata_complete = az.concat(idata, idata_rerun, dim="chain")
idata_complete.posterior.dims["chain"]

6

##  Add groups to InferenceData objects

You can also add new groups to InferenceData objects with the {meth}`~arviz.InferenceData.extend` (if the new groups are already in an InferenceData object) or with {meth}`~arviz.InferenceData.add_groups` (if the new groups are dictionaries or `xarray.Dataset` objects).

In [23]:
rng = np.random.default_rng(3)
idata.add_groups(
    {"predictions": {"obs": rng.normal(size=(4, 500, 2))}}, 
    dims={"obs": ["new_school"]}, 
    coords={"new_school": ["Essex College", "Moordale"]}
)
idata

Inference data with groups:
	> posterior
	> posterior_predictive
	> predictions
	> sample_stats
	> prior
	> observed_data